In [1]:
import pandas as pd
import numpy as np
import csv
import math
import matplotlib.pyplot as plt
from pyts.image import MarkovTransitionField
import warnings
import os

print('donee')

donee


In [3]:
warnings.filterwarnings('ignore') 
#SVC(kernel = 'rbf', gamma = 1e-3, C = 100), day_num = 7, WINDOW = 50

def play_sound():
    os.system("printf '\a'")

def tabulate(x, y, f):
    """Return a table of f(x, y). Useful for the Gram-like operations."""
    return np.vectorize(f)(*np.meshgrid(x, y, sparse=True))

def cos_sum(a, b):
    """To work with tabulate."""
    return(math.cos(a+b))

def sin_diff(a, b):
    """To work with tabulate."""
    return(math.sin(a-b))

def create_time_serie(size, time):
    """Generate a time serie of length size and dynamic with respect to time."""
    # Generating time-series
    support = np.arange(0, size)
    serie = np.cos(support + float(time))
    return(t, serie)

class GASF:

    def __init__(self):
        pass
    def transform(self, serie):
        """Compute the Gramian Angular Field of an image"""
        # Min-Max scaling
        min_ = np.amin(serie)
        max_ = np.amax(serie)
        scaled_serie = (2*serie - max_ - min_)/(max_ - min_)

        # Floating point inaccuracy!
        scaled_serie = np.where(scaled_serie >= 1., 1., scaled_serie)
        scaled_serie = np.where(scaled_serie <= -1., -1., scaled_serie)

        # Polar encoding
        phi = np.arccos(scaled_serie)
        # Note! The computation of r is not necessary
        r = np.linspace(0, 1, len(scaled_serie))

        # GAF Computation (every term of the matrix)
        gaf = tabulate(phi, phi, cos_sum)

        return(gaf, phi, r, scaled_serie)
    
class GADF:

    def __init__(self):
        pass
    def transform(self, serie):
        """Compute the Gramian Angular Field of an image"""
        # Min-Max scaling
        min_ = np.amin(serie)
        max_ = np.amax(serie)
        scaled_serie = (2*serie - max_ - min_)/(max_ - min_)

        # Floating point inaccuracy!
        scaled_serie = np.where(scaled_serie >= 1., 1., scaled_serie)
        scaled_serie = np.where(scaled_serie <= -1., -1., scaled_serie)

        # Polar encoding
        phi = np.arccos(scaled_serie)
        # Note! The computation of r is not necessary
        r = np.linspace(0, 1, len(scaled_serie))

        # GAF Computation (every term of the matrix)
        gaf = tabulate(phi, phi, sin_diff)

        return(gaf, phi, r, scaled_serie)

def csv_extract(day_num, filename = "/MERITZ_DATA.csv"):
    '''
    Input: Number of samples, Number of Stocks, filename
    Output: numpy array of (number of samples, number of stocks)
    '''
    df = pd.read_csv(filename)
    sample_size = df['Open'].shape[0] - day_num
    X = np.empty((sample_size - day_num, 0))
    y = np.empty((sample_size - day_num))
    print(y.shape)

    #Feature Processing. Trying to keep in range [-1, 1]
    
    features = ['Open', 'High', 'Low', 'Close', 'Price_5', 'Price_10', 'Price_20',
                'Price_60', 'Price_120', 'Volume', 'Volume_5', 'Volume_10',
                'Volume_20', 'Volume_60', 'Volume_120', 'Volume Ratio', 'OBV',
                'MACD', 'MACD_Signal', 'Momentum', 'Momentum_Signal', 'William_R',
                'William_D', 'Psychology', 'Psychology_Signal', 'MFI',
                'MFI_Signal', 'RSI', 'RSI_Signal']    
    for feature in features:
        if feature in ['Open', 'High', 'Low', 'Close', 'Volume']:
            X_col = df[feature]
        else:
            continue
        X = np.column_stack((X, X_col[0 : sample_size - day_num]))
        
    for n in range(sample_size - day_num):
        close = df['Close']
        pct = 100 * (close[n + day_num] - close[n]) / close[n]
        if pct > 1.:
            y[n] = 1
        else:
            y[n] = 0
        #y[n] = 1 if df['Close'][n + day_num] >= df['Close'][n] else 0
    return X, y

def feature_extract(X, window = 10):
    feature_num = X.shape[1]
    for i in range(window, X.shape[0]):f
        x = X[i - window: i]
        x = x.T.reshape(x.shape[0] * x.shape[1])
        for j in np.arange(0, window*feature_num, window):
            #x[j : j + window] = (x[j : j + window])
            x[j : j + window] = (x[j : j + window])/np.max(x[j : j + window])
        if i == window:
            X_ = x
            continue
        X_ = np.vstack((X_, x))
    return X_

filename = "./Data/HYNIX_DATA.csv"
WINDOW = 30
FEATURE_NUM = 5
feature_list = ['_open', '_high', '_low', '_close', '_volume']
window = WINDOW
feature_num = FEATURE_NUM
day_num = 4
start_lim = 0
X, y = csv_extract(day_num, filename = filename)
X, y = feature_extract(X[start_lim:], window = WINDOW), y[start_lim + WINDOW:]
path = './Data/Grammian3/'
np.savetxt(path + 'labels', y)
for c in range(X.shape[0]):
    mtf = MarkovTransitionField(image_size = WINDOW)
    gasf, gadf = GASF(), GADF()
    for n, j in enumerate(np.arange(0, window*feature_num, window)):
        x = X[c, j : j + window]
        x_gasf = gasf.transform(x)[0]
        feature_name = feature_list[n]
        gasf_name = path + str(c) + feature_name + '_gasf'
        try:
            np.savetxt(gasf_name, x_gasf)
        except Exception:
            os.mkdir(path)
            np.savetxt(gasf_name, x_gasf)
        if n == 0:
            X_gasf_ = x_gasf
        elif n == 1:
            X_gasf_ = np.stack((X_gasf_, x_gasf), axis = -1)
        else:
            X_gasf_ = np.append(X_gasf_, np.expand_dims(x_gasf, axis= -1), axis = -1)

    if c == 0:
        X_gasf = X_gasf_
    elif c == 1:
        X_gasf = np.stack((X_gasf, X_gasf_), axis = 0)
    else:
        X_gasf = np.append(X_gasf, np.array([X_gasf_]), axis = 0 )
#X_tot = np.concatenate((X_gadf, X_gasf, X_mtf), axis = -1)
X_tot = X_gasf
print(X_tot.shape)
np.save(path + 'X_tot', X_tot)


(2992,)
(2962, 30, 30, 5)
